In [ ]:
import numpy as np
import pandas as pd 

import random
import sys
import matplotlib.pyplot as plt 
%matplotlib inline

sys.path.insert(1,'D:\\thesis-main\codes')
import time

from package.loadDb import *
from package.featureSelection_TENSORIZED import *
from package.featureSelection_TEMPORAL_PAGERANK import *
from package.featureSelection_SICE import *

from package.trmf import *
from package.trmf_regressor import *

from package.regressors import *

In [ ]:
dataset_name=
address = "..\\database\\appliances+energy+prediction\\energydata_complete.csv"
data_with_target =load_energydata_complete_dataframe(address)
data_without_target = data_with_target.drop(['Appliances'],axis=1)
target=data_with_target.iloc[:,0]

In [ ]:
data_with_target

In [ ]:

data_with_target.columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming your data is already loaded in a DataFrame called data_with_target
# Example column lists
col1 = ['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
        'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9',
        'RH_9']
col2 = ['T_out', 'RH_out', 'Windspeed', 'Visibility',
        'Tdewpoint', 'rv1', 'rv2']
col3 = ['Appliances', 'lights', 'Press_mm_hg']
col4=['Appliances','RH_5']

# Function to create scatter plots
def plot_columns(column_list, title):
    plt.figure(figsize=(14, 8))
    for column in column_list:
        plt.scatter(data_with_target.index, data_with_target[column], label=column)
    plt.xlabel('Date')
    plt.ylabel('Values')
    plt.title(f'Time Series Plot of {title}')
    plt.legend(loc='best', bbox_to_anchor=(1, 1))
    plt.grid(True)
    plt.show()

# Plot for col1
plot_columns(col1, 'Temperature and Humidity')

# Plot for col2
plot_columns(col2, 'Environmental Factors')

# Plot for col3
plot_columns(col3, 'Appliances, Lights, and Pressure')
plot_columns(col4, 'Appliances, RH_5')



In [ ]:
address = "..\\database\\appliances+energy+prediction\\energydata_complete.csv"
data_with_target =load_energydata_complete_dataframe(address)
data_without_target = data_with_target.drop(['Appliances'],axis=1)
target=data_with_target.iloc[:,0]

In [ ]:
#segment sizes
interval=600
testSize=int(.2*interval)
#energy of pca to compute number of components
tol=.89
testSize

In [ ]:
# trmf parameters..............................................
lags = [1,30]
# K = 4######### changes besed on pca energy
lambda_f = 1.
lambda_x = 1
lambda_w = 1.
alpha = 1000.
eta = 1.
max_iteration=10000
# num_comp=K

In [ ]:
num_comp_list=[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 4,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1]

In [ ]:
i

In [ ]:
from itertools import product

# Define parameter grids
lags_list = [[1], [1, 30]]
lambda_f_list = [0.1, 1., 10.]
lambda_x_list = [0.1, 1., 10.]
lambda_w_list = [0.1, 1., 10.]
alpha_list = [100, 1000, 10000]
eta_list = [0.1, 1., 10.]

# Store all RMSE results for each segment and parameter combination
results = []

counter = 0


# Iterate over all combinations of parameters except K
for lags, lambda_f, lambda_x, lambda_w, alpha, eta in product(lags_list, lambda_f_list, lambda_x_list, lambda_w_list, alpha_list, eta_list):
    i = 0
    for j in range(data_without_target.shape[0] // interval):
        start_index = j * interval
        end_index = start_index + interval

        K = num_comp_list[i]  # Each segment has different k
        data = data_without_target.to_numpy().T
        data = data[:, start_index:end_index]
        target_cut = target.iloc[start_index:end_index]

        T_train = data.shape[1]
        st = time.time()  # Start timer
        data_normalized = normalized_data(data, T_train, 0, normalize=True)

        model = trmf(lags, K, lambda_f, lambda_x, lambda_w, alpha, eta)
        model.fit(data_normalized, max_iter=max_iteration)
        et = time.time()  # End timer
        elapsed_time = et - st
        runtime.append(elapsed_time)

        # Get reduced data
        new_features = model.X.T
        rmse = xgboost_reg_error(new_features, target_cut, testSize)
        trmf_rmse_list.append(rmse)

        # Store the parameters and RMSE for this segment
        results.append({
            'segment': j,
            'K': K,
            'lags': lags,
            'lambda_f': lambda_f,
            'lambda_x': lambda_x,
            'lambda_w': lambda_w,
            'alpha': alpha,
            'eta': eta,
            'rmse': rmse,
            'runtime': elapsed_time
        })

        i += 1
        print(f"Iteration {i}, Segment {j}, K={K}, RMSE={rmse}, Time={elapsed_time}")
        print("----------------------------------------------------------------------------")

# Convert results to a DataFrame for easier analysis and visualization
import pandas as pd
results_df = pd.DataFrame(results)

# Example: Save results to a CSV file for later analysis
results_df.to_csv('trmf_tuning_results.csv', index=False)

# Now you can analyze or visualize the results


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure 'segment' is treated as a string or categorical variable
results_df['segment'] = results_df['segment'].astype(str)

# Convert list-type columns to tuples
results_df['lags'] = results_df['lags'].apply(lambda x: tuple(x) if isinstance(x, list) else x)

# Create a unique identifier for each parameter set
results_df['param_id'] = results_df.groupby(['lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta']).ngroup() + 1

# Get unique segments
unique_segments = results_df['segment'].unique()

# Plot size customization
plt.rcParams.update({'figure.figsize': (100, 30)})

# Iterate over each segment and create a separate plot
for segment in unique_segments:
    # Filter the DataFrame for the current segment
    segment_df = results_df[results_df['segment'] == segment]

    # Create a bar plot for the current segment
    plt.figure(figsize=(14, 10))
    sns.barplot(
        x='param_id', 
        y='rmse', 
        data=segment_df
    )

    # Set plot labels and title
    plt.xlabel('Parameter Set ID')
    plt.ylabel('RMSE')
    plt.title(f'RMSE for Segment {segment}')

    # Rotate x-axis labels if needed for better readability
    plt.xticks(rotation=0)

    # Show the plot
    plt.show()

# Create a separate legend figure to map numbers to parameter sets
unique_params = results_df.drop_duplicates('param_id').sort_values('param_id')
plt.figure(figsize=(12, len(unique_params) * 0.5))  # Adjust the figure size based on the number of sets

# Plot the mapping of parameter set ID to actual parameters
plt.table(cellText=unique_params[['param_id', 'lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta']].values,
          colLabels=['ID', 'lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta'],
          cellLoc = 'center', 
          loc='center')

plt.axis('off')  # Hide the axis
plt.title('Mapping of Parameter Set ID to Parameter Values')
plt.show()


In [ ]:
results_df['params'] = results_df[['lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta']].apply(lambda x: ', '.join(map(str, x)), axis=1)


In [ ]:

mean_rmse_df = results_df.groupby([ 'lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta'])['rmse'].mean().reset_index()

mean_rmse_df.sort_values(by=['lags', 'lambda_f', 'lambda_x', 'lambda_w', 'alpha', 'eta'], ascending=[True, True, True, True, True, True]).to_excel("rmse_trmf_parameter_tuning.xlsx")

In [103]:
import pandas as pd
import numpy as np
from tsflex.features import FeatureCollection, FeatureDescriptor, FuncWrapper
from sklearn.feature_selection import SelectKBest, f_regression

# Load your data
address = "..\\database\\appliances+energy+prediction\\energydata_complete.csv"
data_with_target = pd.read_csv(address)
data_with_target = data_with_target.reset_index()
data_with_target = data_with_target.rename(columns={"index": "id"})
# data_with_target=data_with_target.loc[:10]
# Separate features and target
data_without_target = data_with_target.drop(['Appliances'], axis=1)
target = data_with_target['Appliances']

data_without_target


,id,date,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0,2016-01-11 17:00:00,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,...,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,1,2016-01-11 17:10:00,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,...,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,2,2016-01-11 17:20:00,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,...,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,3,2016-01-11 17:30:00,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,4,2016-01-11 17:40:00,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,19730,2016-05-27 17:20:00,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,...,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,19731,2016-05-27 17:30:00,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,...,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,19732,2016-05-27 17:40:00,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,...,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,19733,2016-05-27 17:50:00,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,...,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


In [92]:
from tsfresh import extract_features, select_features,feature_selection
X_extracted = extract_features(data_without_target, column_id='id', column_sort='date',pivot=False)

Feature Extraction: 100%|██████████| 9/9 [00:04<00:00,  1.96it/s]


In [102]:
target.unique()

array([  60,   50,   70,  230,  580,  430,  250,  100,   90,   80,  140,
        120,  190,  110,  400,  390,  240,   40,   30,  310,  380,  370,
         20,  260,  500,  450,  220,  170,  290,  130,  200,  210,  180,
        350,  300,  340,  150,  330,  420,  520,  360,  270,  550,  690,
        620,  610,  490,  410,  320,  740,  910,  510,  790,  460,  160,
        480,  280,  560, 1080,  750,  470,  440,  800,  540,  530,  660,
        630,  600,   10,  570,  830, 1070,  890,  590,  850,  710,  680,
        670,  780,  640,  650,  700,  770,  720,  760,  860,  880,  730,
        900,  820,  870,  840], dtype=int64)

In [138]:
data_without_target.iloc[:10,5]

0    19.20
1    19.20
2    19.20
3    19.20
4    19.20
5    19.20
6    19.20
7    19.20
8    19.20
9    19.23
Name: T2, dtype: float64

In [139]:
from tsfresh import  select_features,feature_selection

X_selected = select_features(data_without_target.iloc[:30,2:], target.iloc[:30])


In [140]:
X_selected

,T9,Press_mm_hg,Tdewpoint,T2,RH_4,RH_7,T1,T3,RH_out,T7,RH_3,T6
0,17.033333,733.500000,5.300000,19.200000,45.566667,41.626667,19.890000,19.790000,92.000000,17.200000,44.730000,7.026667
1,17.066667,733.600000,5.200000,19.200000,45.992500,41.560000,19.890000,19.790000,92.000000,17.200000,44.790000,6.833333
2,17.000000,733.700000,5.100000,19.200000,45.890000,41.433333,19.890000,19.790000,92.000000,17.200000,44.933333,6.560000
3,17.000000,733.800000,5.000000,19.200000,45.723333,41.290000,19.890000,19.790000,92.000000,17.133333,45.000000,6.433333
4,17.000000,733.900000,4.900000,19.200000,45.530000,41.230000,19.890000,19.790000,92.000000,17.200000,45.000000,6.366667
5,17.000000,734.000000,4.800000,19.200000,45.730000,41.260000,19.890000,19.790000,92.000000,17.133333,44.933333,6.300000
6,17.000000,734.100000,4.700000,19.200000,45.790000,41.200000,19.890000,19.790000,92.000000,17.133333,44.900000,6.263333
7,17.000000,734.166667,4.683333,19.200000,45.863333,41.200000,19.856667,19.730000,91.833333,17.100000,44.900000,6.190000
8,17.000000,734.233333,4.666667,19.200000,45.790000,41.400000,19.790000,19.730000,91.666667,17.166667,44.790000,6.123333
9,17.000000,734.300000,4.650000,19.230000,46.096667,41.500000,19.856667,19.790000,91.500000,17.200000,44.863333,6.190000


In [136]:
from tsfresh.feature_selection.relevance import calculate_relevance_table
relevance_table = calculate_relevance_table(data_without_target.iloc[:30, 2:], data_without_target.iloc[:30, 5])

# Select the top 7 features
top_7_features = relevance_table.sort_values(by="p_value").head(7).index
X_selected = data_without_target[top_7_features]

In [137]:
X_selected

,T2,Tdewpoint,Press_mm_hg,T1,T3,RH_out,RH_9
0,19.200000,5.300000,733.5,19.890000,19.790000,92.000000,45.5300
1,19.200000,5.200000,733.6,19.890000,19.790000,92.000000,45.5600
2,19.200000,5.100000,733.7,19.890000,19.790000,92.000000,45.5000
3,19.200000,5.000000,733.8,19.890000,19.790000,92.000000,45.4000
4,19.200000,4.900000,733.9,19.890000,19.790000,92.000000,45.4000
...,...,...,...,...,...,...,...
19730,25.890000,13.333333,755.2,25.566667,27.200000,55.666667,46.7900
19731,25.754000,13.300000,755.2,25.500000,27.133333,56.000000,46.7900
19732,25.628571,13.266667,755.2,25.500000,27.050000,56.333333,46.7900
19733,25.414000,13.233333,755.2,25.500000,26.890000,56.666667,46.8175


In [ ]:
print(X_extracted.dtypes)


In [36]:
X_extracted = X_extracted.fillna(X_extracted.mean())

In [ ]:
feature_selection.significance_tests.target_real_feature_real_test(data_without_target,target)

In [66]:
import pandas as pd
from tsfresh.feature_selection.significance_tests import target_real_feature_real_test


# Initialize a dictionary to store p-values
p_values = {}

# Perform significance test for each feature
for feature in data_without_target.columns[2:]:
    x = data_without_target[feature]
    p_value = target_real_feature_real_test(x, target)
    p_values[feature] = p_value

# Convert p-values to a DataFrame for easy viewing
p_values_df = pd.DataFrame(list(p_values.items()), columns=['Feature', 'P-value']).sort_values(by="P-value")

p_values_df


,Feature,P-value
12,RH_6,0.007187
21,RH_out,0.011774
3,T2,0.014098
26,rv2,0.046113
24,Tdewpoint,0.046113
20,Press_mm_hg,0.046113
25,rv1,0.046113
4,RH_2,0.079095
13,T7,0.097306
8,RH_4,0.098597
